In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
url = 'http://archive.ics.uci.edu/ml/' \
      'machine-learning-databases/auto-mpg/auto-mpg.data'

column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower',
        'Weight', 'Acceleration', 'Model Year', 'Origin']

numerical_columns = ['Cylinders', 'Displacement', 'Horsepower',
        'Weight', 'Acceleration']

df = pd.read_csv(url,names=column_names, sep=' ', comment='\t',na_values = "?",skipinitialspace=True)

# Drop the NA rows
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)    # assigning a new default integer index to the DataFrame

# Convert The Model Year column
boundries = torch.tensor([73,76,79])
v = torch.tensor(df['Model Year'].values)
df['Model Year'] = torch.bucketize(v,boundries,right=True)

# One-Hot-Encoding for the 'Original' column
origin_encoded = one_hot(torch.tensor((df['Origin']-1).values),num_classes=df['Origin'].nunique())[:,1:]
data = torch.cat([torch.tensor(df.drop(columns='Origin').values),origin_encoded],dim=1)
column_names_encoded = ['MPG', 'Cylinders', 'Displacement', 'Horsepower',
        'Weight', 'Acceleration', 'Model Year', 'Europe','Japan']
df = pd.DataFrame(data,columns=column_names_encoded)

# Split our data
df_train, df_test = train_test_split(df,test_size=0.8,random_state=123)

# Sdandarize it
for col in numerical_columns:
    mean = df_train[col].mean()
    std = df_train[col].std()
    df_train[col] = (df_train[col] - mean) / std
    df_test[col] = (df_test[col] - mean) / std

# Devide the data
X_train = torch.tensor(df_train.iloc[:,1:].values).float()
X_test = torch.tensor(df_test.iloc[:,1:].values).float()
y_train = torch.tensor(df_train.iloc[:,0].values).float()
y_test = torch.tensor(df_test.iloc[:,0].values).float()

df_train.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan
3,16.0,1.629454,1.192495,1.373011,0.629638,-1.470879,0.0,0.0,0.0
56,24.0,-0.779976,-0.748803,-0.226722,-0.812878,-0.048960,0.0,0.0,1.0
304,28.8,0.424739,-0.138971,0.354999,-0.416967,-1.755263,3.0,0.0,0.0
233,26.0,-0.779976,-0.911425,-0.808443,-0.829115,1.047949,2.0,0.0,1.0
51,30.0,-0.779976,-1.002900,-0.779357,-1.078901,-0.455222,0.0,1.0,0.0


In [3]:
data_train = DataLoader(dataset=TensorDataset(X_train,y_train), shuffle=True, batch_size=8)

In [ ]:
class PFECModel(nn.Module):

    def __init__(self,n_input,n_output):
        super(PFECModel, self).__init__()    # Call the parent class's __init__()
        self.n_input = n_input
        self.n_output = n_output
        self.Seq = nn.Sequential(
            nn.Linear(self.n_input,20),
            nn.ReLU(),
            nn.Linear(20,10),
            nn.ReLU(),
            nn.Linear(10,self.n_output)
        )

    def forward(self,x):
        return self.Seq(x)
    
model = PFECModel(8,1)

        

In [12]:
loss_fn = nn.MSELoss()
lr = 0.01
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

In [ ]:
losses = []
for (x_batch,y_batch) in data_train:
    pred = model(x_batch)
    loss = loss_fn(y_batch,pred)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss.item())





tensor(0.1132, grad_fn=<SelectBackward0>)